In [1]:
%load_ext autoreload
%autoreload 2

import tempfile
import os
import csv

import tensorflow as tf
import numpy as np

import tensorflow.keras.utils
from tensorflow import keras
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import itertools
import soundfile as sf
import librosa
import math
import time
from prune import prune
import tensorflow_model_optimization as tfmot

%load_ext tensorboard
print("done importing")

2023-05-07 18:14:35.279161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


done importing


In [2]:
# load the model first
# model = hub.load('https://tfhub.dev/google/humpback_whale/1')
m = tf.keras.Sequential([hub.KerasLayer("model", trainable=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
m.build((None, 39124,1))
m.summary()

2023-05-07 18:14:51.941270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,39124,1]
	 [[{{node inputs}}]]
2023-05-07 18:14:52.105807: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,39124,1]
	 [[{{node Placeholder}}]]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 23,538,755
Trainable params: 23,477,443
Non-trainable params: 61,312
_________________________________________________________________


In [19]:
i=0
whales_identified=0
whale_samples=0
other_samples_identified=0
other_samples=0
directory_mn='./clips/mn/'
directory_not_mn='./clips/not_mn/'

start_time=time.time()
for file in os.listdir(directory_mn):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
    # try opening the file
    try:
        waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_mn+filename))
    except:
        print("error encountered while processing "+filename+", continuing with inference")
        continue
    waveform = tf.expand_dims(waveform,0)  # makes a batch of size 1
    try:
        score = m.predict(waveform)
    except:
        print('could not run inference on ' + filename)
        continue
    if len(score[0]) == 0:
        continue
    score = score[0][0]
    if filename[0:2]=='Mn':
        whale_samples+=1
        if score >= 0.5:
            whales_identified+=1
    else:
        other_samples+=1
        if score < 0.5:
            other_samples_identified+=1
    i+=1
    
for file in os.listdir(directory_not_mn):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
        
    # try opening the file
    try:
        waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(directory_not_mn+filename))
    except:
        print("error encountered while processing "+filename+", continuing with inference")
        continue
    waveform = tf.expand_dims(waveform,0)
    try:
        score = m.predict(waveform)
    except:
        print('could not run inference on ' + filename)
        continue
    if len(score[0]) == 0:
        continue
    score = score[0][0]
    if filename[0:2]=='Mn':
        whale_samples+=1
        if score >= 0.5:
            whales_identified+=1
    else:
        other_samples+=1
        if score < 0.5:
            other_samples_identified+=1
    i+=1
        

end_time = time.time()
print('done processing data, elapsed time: '+str(end_time-start_time))

1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 86ms/step
could not run inference on Mn-78670550.0-Hawaii_K_14_121216_190000.df20.x.wav
1/1 [==============================] - 0s 60ms/step


2023-05-07 17:15:31.946070: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 71ms/step
could not run inference on Mn-78479588.0-Hawaii_K_14_121216_190000.df20.x.wav
1/1 [==============================] - 0s 77ms/step


2023-05-07 17:15:42.716728: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


...

1/1 [==============================] - 0s 65ms/step
could not run inference on Other-120165645.0-Hawaii_K_20_320_150706_103140_df32.x.wav
1/1 [==============================] - 0s 64ms/step


2023-05-07 17:20:57.250531: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 75ms/step
could not run inference on Other-118523609.0-Hawaii_K_20_320_150801_174730_df32.x.wav
1/1 [==============================] - 0s 65ms/step


2023-05-07 17:20:57.607332: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices = 0 is not in [0, 0)
	 [[{{node GatherV2}}]]


1/1 [==============================] - 0s 70ms/step
done processing data, elapsed time: 365.90700817108154


In [29]:
print('model accuracy: ' + str((whales_identified+other_samples_identified)*1.0/i))
print('proportion of whale samples to total samples: '+str((whale_samples*1.0)/i))
print('total samples: '+ str(i))

model accuracy: 0.46102598267821454
proportion of whale samples to total samples: 0.9733510992671552
total samples: 3002


In [10]:
'''
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    # if this isn't a .wav file, skip
    if filename[-4:] != '.wav':
        continue
    if filename[0:2]=='Mn':
        os.system('mv ' + directory + filename + ' ./clips/mn')
    else:
        os.system('mv ' + directory + filename + ' ./clips/not_mn')

'''
train_ds, val_ds = tensorflow.keras.utils.audio_dataset_from_directory(
    directory='./clips',
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=39124,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Found 3012 files belonging to 2 classes.
Using 2410 files for training.
Using 602 files for validation.

label names: ['mn' 'not_mn']


In [38]:
def apply_pruning_to_dense(layer):
    if isinstance(layer, tf.keras.layers.Dense):
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer

m_for_pruning = tf.keras.models.clone_model(
    m,
    clone_function=apply_pruning_to_dense
)

def get_prunable_weights(self):
    return self.get_weights()

# hub.keras_layer.KerasLayer.get_prunable_weights = get_prunable_weights
# print(m_for_pruning.layers[0])
# for layer in m_for_pruning.layers:
    # layer.get_prunable_weights = get_prunable_weights.__get__(layer)

pruned_m=prune(m_for_pruning, train_ds, val_ds)

m.summary()
pruned_m.summary()

2023-05-07 22:19:38.643723: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,?,?]
	 [[{{node Placeholder}}]]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 23,538,755
Trainable params: 23,477,443
Non-trainable params: 61,312
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 prune_low_magnitude_dense (  (None, 1)                5         
 PruneLowMagnitude)                                              
                   

In [13]:
m_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1)                 23538753  
                                                                 
 prune_low_magnitude_dense (  (None, 1)                5         
 PruneLowMagnitude)                                              
                                                                 
Total params: 23,538,758
Trainable params: 23,477,443
Non-trainable params: 61,315
_________________________________________________________________
